# 2020.11.20 空品小時值爬蟲

1. 確認目標網站([https://data.epa.gov.tw/dataset/aqx_p_15](https://data.epa.gov.tw/dataset/aqx_p_15))，評估爬蟲方法(爬蟲下載) (2020.11.20)
2. 撰寫爬蟲腳本
3. 存入SQLite
4. 檢查資料品質，特殊處理，變數製作 如8小時移動平均

- 目標網站:
    - 空品小時值(包含溫度、降雨、NMHC、THC): https://data.epa.gov.tw/dataset/aqx_p_15
        - 2020.11月 只有29天份資料
    - AQI小時值(有算各污染物的平均值): https://data.epa.gov.tw/dataset/aqx_p_488
    - NMHC: https://data.epa.gov.tw/dataset/aqx_p_17
    - THC: https://data.epa.gov.tw/dataset/aqx_p_18
    - 空品日均值: https://data.epa.gov.tw/dataset/aqx_p_19

## 1. 取得檔案名稱列表與下載連結

In [1]:
import requests as rs
from bs4 import BeautifulSoup as BS
import urllib
from lxml import etree

from glob import glob
from zipfile import ZipFile
import io

from tqdm import tqdm
from datetime import date
from time import process_time

from sqlalchemy import create_engine

import sys
sys.path.append('G:\我的雲端硬碟\python')
from polib.CsvEngn import *
import numpy as np

In [2]:
dicHeaders = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"}

url_object = 'https://data.epa.gov.tw/dataset/aqx_p_15' # 空品小時值
# url_object = 'https://data.epa.gov.tw/dataset/aqx_p_488' # AQI
# url_object = 'https://data.epa.gov.tw/dataset/aqx_p_17' # NMHC
# url_object = 'https://data.epa.gov.tw/dataset/aqx_p_18' # THC

In [3]:
sssn = rs.session()
sssn.headers = dicHeaders
sssn.stream = True
sssn.verify = False

In [4]:
# 關閉連線警告
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

In [5]:
response = sssn.get(url_object, verify=False)
# response.status_code # 回傳狀態 200,404,...
if response.status_code == 200:
    print('>> 網頁訪問成功!!')

>> 網頁訪問成功!!


In [6]:
# 解析內容 (轉為string)
content = response.content.decode()
# 把string轉為hmtl node tree，回傳根節點
html = etree.HTML(content)

In [7]:
# content

暫時性輸出最新月份資料

In [8]:
# # title = html.xpath("//*[@id='dataset-resources']/div/table/tbody/tr[*]/td[1]/a/span//@title")
# title = html.xpath("//*[@id='dataset-resources']/div/table/tbody/tr[*]/td[1]/a/span/text()")
# title_lst = [' '.join(t.split()) for t in title]
# title_pic = title_lst[0] # 選取要加入的月份
# print(title_pic)

# download = html.xpath("//*[@id='dataset-resources']/div/table/tbody/tr[*]/td[2]/a//@href")
# download = download[:1] # 選取要加入的月份

# date_col_name = "DataCreationDate"

In [9]:
# df_result = pd.DataFrame()
# # zip url 讀取
# for zip_file_url in download:
#     r = rs.get(zip_file_url,verify=False)
#     z = ZipFile(io.BytesIO(r.content))
#     print(f">> {zip_file_url.split('/')[-1]}")
    
#     # zip中每個csv 讀取
#     for f_name in tqdm(sorted(z.namelist())):
#         df_tmp = pd.read_csv(z.open(f_name), parse_dates=[date_col_name])
#         df_tmp_s = df_tmp.reset_index()
#         df_result = pd.concat([df_result,df_tmp_s])

In [10]:
# formtoCsv(df_result, "AQI_2月至25_am9資料")

暫時性輸出最新月份資料_ OVER

In [11]:
# title = html.xpath("//*[@id='dataset-resources']/div/table/tbody/tr[*]/td[1]/a/span//@title")
title = html.xpath("//*[@id='dataset-resources']/div/table/tbody/tr[*]/td[1]/a/span/text()")
title_lst = [' '.join(t.split()) for t in title]
# title_lst
pic_end_num = 43
title_pic = title_lst[1:pic_end_num] # 選取要加入的月份
title_pic

['空氣品質監測小時值(一般污染物,每日更新) (2021/06)',
 '空氣品質監測小時值(一般污染物,每日更新) (2021/05)',
 '空氣品質監測小時值(一般污染物,每日更新) (2021/04)',
 '空氣品質監測小時值(一般污染物,每日更新) (2021/03)',
 '空氣品質監測小時值(一般污染物,每日更新) (2021/02)',
 '空氣品質監測小時值(一般污染物,每日更新) (2021/01)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/12)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/11)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/10)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/09)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/08)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/07)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/06)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/05)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/04)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/03)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/02)',
 '空氣品質監測小時值(一般污染物,每日更新) (2020/01)',
 '空氣品質監測小時值(一般污染物,每日更新) (2019/12)',
 '空氣品質監測小時值(一般污染物,每日更新) (2019/11)',
 '空氣品質監測小時值(一般污染物,每日更新) (2019/10)',
 '空氣品質監測小時值(一般污染物,每日更新) (2019/09)',
 '空氣品質監測小時值(一般污染物,每日更新) (2019/08)',
 '空氣品質監測小時值(一般污染物,每日更新) (2019/07)',
 '空氣品質監測小時值(一般污染物,每日更新) (2019/06)',
 '空氣品質監測小時值(一般污染物,每日更新) (2019/05)',
 '空氣品質監測小時值(一般污染物,每日更新) (2019/04)',
 '空氣品質監測小時值(一般污染物,每日更新) (201

In [12]:
download = html.xpath("//*[@id='dataset-resources']/div/table/tbody/tr[*]/td[2]/a//@href")
download = download[1:pic_end_num] # 選取要加入的月份
# download

可參考這篇往下做: https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url

In [13]:
# 建立資料庫連線
db_name = "air_monitor_new"

table_name = "air_monitor"
# table_name = "AQI"


db_path = f'D:\\SQLiteDBs\\{db_name}.db'

db_path_ready = db_path.replace('\/' , '/')
db_engine = create_engine(f'sqlite:///{db_path_ready}', echo=False)
print(f'>>> DB 成功連線 {db_path} Table: {table_name}!!\n')

>>> DB 成功連線 D:\SQLiteDBs\air_monitor_new.db Table: air_monitor!!



In [14]:
if table_name in ["air_monitor"]:
    case = "M"
    date_col_name = "MonitorDate"
    
if table_name in ["AQI"]:
    case = "D"
    date_col_name = "DataCreationDate"

In [15]:
# zip url 讀取
for zip_file_url in download:
    r = rs.get(zip_file_url,verify=False)
    z = ZipFile(io.BytesIO(r.content))
    print(f">> {zip_file_url.split('/')[-1]}")
    
    # zip中每個csv 讀取
    for f_name in tqdm(sorted(z.namelist())):
        df_tmp = pd.read_csv(z.open(f_name), parse_dates=[date_col_name])
        
        # 資料處理 轉置為以小時為row
        if case=="M":
            df_tmp.set_index(df_tmp.columns.to_list()[:7], inplace=True)
            df_tmp_s = df_tmp.stack().reset_index()
            df_tmp_s.rename(columns={0:'MonitorValue'}, inplace=True)
            df_tmp_s.loc[:,date_col_name] = df_tmp_s.loc[:,[date_col_name, 'level_7']].apply(lambda row: row[date_col_name].replace(hour=int(row['level_7'][-2:]\
                                                                                                        if type(row[date_col_name]) is pd.Timestamp \
                                                                                                        else np.nan)), axis=1)
            df_tmp_s.drop(columns=['level_7'], inplace=True)
            
        if case=="D":
            df_tmp_s = df_tmp.reset_index()
        
        
        # 存入資料庫
        df_tmp_s.to_sql(table_name, db_engine, if_exists='append', index=False)
        

print('>>> 資料儲存完成!!\n\n')
del df_tmp
del df_tmp_s

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

>> aqx_p_15_2021-06.zip


  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

>> aqx_p_15_2021-05.zip


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

...

>> aqx_p_15_2018-02.zip


  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

>> aqx_p_15_2018-01.zip


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:48<00:00,  1.56s/it]

>>> 資料儲存完成!!




### 建立索引 

In [21]:
# # 建立資料庫連線
# db_path = r'D:\SQLiteDBs\air_monitor_new.db'

# db_path_ready = db_path.replace('\/' , '/')
# db_engine = create_engine(f'sqlite:///{db_path_ready}', echo=False)
# print('>>> DB 成功連線!!\n')

# sql_cmd = f"CREATE INDEX {date_col_name} on {table_name} ({date_col_name}); "

# print(f'>> 開始執行 SQL:{sql_cmd}\n')
# a= process_time()
# pd.read_sql(sql=sql_cmd, con=db_engine)
# b = process_time()
# print(f'>> SQL 讀取完成!! 耗時:{b-a}秒\n{df.info()}')

## 2. 從SQLite中讀取

In [17]:
# 建立資料庫連線
db_path = r'D:\SQLiteDBs\air_monitor_new.db'

db_path_ready = db_path.replace('\/' , '/')
db_engine = create_engine(f'sqlite:///{db_path_ready}', echo=False)
print('>>> DB 成功連線!!\n')

>>> DB 成功連線!!



In [5]:
# df['ItemEngName'].unique()

### (參數) 選擇處理項目與時間

In [3]:
# select_item

In [18]:
# 1.選擇項目
table_name = "air_monitor"
# table_name = "AQI"
# table_name = "NMHC"
# table_name = "THC"

# select_item = "O3"
# select_item = ["PM2.5", "PM2.5_AVG", "O3"]
# select_item = ["O3", "O3_8hr", "CO","CO_8hr", "NOx","NO","NO2","SO2","PM10","WindSpeed","WindDirec","SO2_AVG","PM2.5","PM2.5_AVG","PM10_AVG"]

# 2.選擇時間_從...至今
# date_after='2009-12-31'  #範例格式: 2017-12-31
# date_after='2016-12-31'  #範例格式: 2017-12-31  (計算冷熱季需要更長的時間，目前用近10年: 2010~2020)
# date_after='2000-1-1' # 全都要
# date_after='2021-03-01' # test
date_after='2018-01-01' # test

In [19]:
if table_name in ["air_monitor"]:
    case = "M"
    date_col_name = "MonitorDate"
    
if table_name in ["AQI"]:
    case = "D"
    date_col_name = "DataCreationDate"

### SQL指令

In [6]:
# df['ItemName'].unique()

In [20]:
# sql 指令
# sql_cmd = "SELECT * FROM air_monitor LIMIT 1000" #測試模式
# sql_cmd = f"SELECT * FROM {table_name} WHERE ItemEngName ='{select_item}' and Date(MonitorDate)>='{date_after}' "
sql_cmd = f"SELECT * FROM {table_name} WHERE Date({date_col_name})>='{date_after}' "
sql_cmd = f"SELECT * FROM {table_name} "

print(f'>> 開始執行 SQL:{sql_cmd}\n')
a= process_time()
df = pd.read_sql(sql=sql_cmd, con=db_engine)
b = process_time()
print(f'>> SQL 讀取完成!! 耗時:{b-a}秒\n{df.info()}')

>> 開始執行 SQL:SELECT * FROM air_monitor WHERE Date(MonitorDate)>='2018-01-01' 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38115683 entries, 0 to 38115682
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   SiteId        float64
 1   SiteName      object 
 2   ItemId        int64  
 3   ItemName      object 
 4   ItemEngName   object 
 5   ItemUnit      object 
 6   MonitorDate   object 
 7   MonitorValue  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 2.3+ GB
>> SQL 讀取完成!! 耗時:816.0秒
None


In [19]:
# formtoCsv(df,"AQI_2021M3")

### table_name = "air_monitor" 資料轉置  1 row 1 項目 (pivot)

In [23]:
df_pivot = df.pivot(index=["SiteName", "MonitorDate"], columns='ItemEngName', values='MonitorValue').reset_index()

In [30]:
df_pivot.loc[:,"MonitorDate"] = pd.to_datetime(df_pivot.loc[:,"MonitorDate"], format='%Y-%m-%d %H:%M:%S.%f')

In [31]:
df_pivot["MonitorDate"].dt.year.value_counts()

2018    672723
2020    671204
2019    670346
2021    336307
Name: MonitorDate, dtype: int64

資料清理

In [33]:
df_pivot.head(1)

ItemEngName,SiteName,MonitorDate,AMB_TEMP,CH4,CO,CO2,NMHC,NO,NO2,NOx,...,RAINFALL,RAIN_COND,RAIN_INT,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,三義,2018-01-01,11.6,NaN,0.26,NaN,NaN,0.8,4.7,5.5,...,NaN,0,0,92,2.3,NaN,24,24,3.9,3.8


In [35]:
df_pivot.columns

Index(['SiteName', 'MonitorDate', 'AMB_TEMP', 'CH4', 'CO', 'CO2', 'NMHC', 'NO',
       'NO2', 'NOx', 'O3', 'PH_RAIN', 'PM10', 'PM2.5', 'RAINFALL', 'RAIN_COND',
       'RAIN_INT', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC', 'WIND_SPEED',
       'WS_HR'],
      dtype='object', name='ItemEngName')

In [41]:
select_item = [
#     'SiteName', 
#     'MonitorDate', 
    'AMB_TEMP', 
    'CH4', 
    'CO', 
    'CO2', 
    'NMHC', 
    'NO',
    'NO2', 
    'NOx', 
    'O3', 
#     'PH_RAIN', 
    'PM10', 
    'PM2.5', 
#     'RAINFALL', 
#     'RAIN_COND',
#     'RAIN_INT', 
#     'RH', 
    'SO2', 
    'THC', 
#     'WD_HR', 
#     'WIND_DIREC', 
#     'WIND_SPEED',
#     'WS_HR'
]

# 清理指定欄位的雜值
def cleaner(v):
    try:
        v=int(float(v))
        if v<0:
            return np.nan
        return v
    except:
            return np.nan

print(select_item)
for col in tqdm(select_item):
    df_pivot.loc[:, col] = df_pivot.loc[:, col].apply(cleaner)

  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

['AMB_TEMP', 'CH4', 'CO', 'CO2', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM2.5', 'SO2', 'THC']


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:30<00:00,  2.32s/it]


In [37]:
df_pivot.loc[:, select_item].describe()

ItemEngName,AMB_TEMP,CH4,NMHC,THC
count,2.312599e+06,1.138394e+06,1.138392e+06,1.138399e+06
mean,2.357746e+01,1.319682e+00,2.085398e-03,1.627682e+00
std,5.422135e+00,4.724244e-01,5.227665e-02,5.047381e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
25%,2.000000e+01,1.000000e+00,0.000000e+00,1.000000e+00
50%,2.400000e+01,1.000000e+00,0.000000e+00,2.000000e+00
75%,2.800000e+01,2.000000e+00,0.000000e+00,2.000000e+00
max,9.100000e+01,8.000000e+00,1.000000e+01,1.200000e+01


In [7]:
# df.loc[:,"DataCreationDate"] = pd.to_datetime(df.loc[:,"DataCreationDate"])
# df_pivot.loc[:,"date"] = df_pivot.loc[:,"DataCreationDate"].dt.date

In [38]:
date_col = "MonitorDate"
df_pivot.sort_values(by=['SiteName',date_col], inplace=True)
df_pivot.drop_duplicates(keep="first", inplace=True)
df_pivot.reset_index(drop=True, inplace=True)

In [40]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2350580 entries, 0 to 2350579
Data columns (total 24 columns):
 #   Column       Dtype         
---  ------       -----         
 0   SiteName     object        
 1   MonitorDate  datetime64[ns]
 2   AMB_TEMP     float64       
 3   CH4          float64       
 4   CO           object        
 5   CO2          object        
 6   NMHC         float64       
 7   NO           object        
 8   NO2          object        
 9   NOx          object        
 10  O3           object        
 11  PH_RAIN      object        
 12  PM10         object        
 13  PM2.5        object        
 14  RAINFALL     object        
 15  RAIN_COND    object        
 16  RAIN_INT     object        
 17  RH           object        
 18  SO2          object        
 19  THC          float64       
 20  WD_HR        object        
 21  WIND_DIREC   object        
 22  WIND_SPEED   object        
 23  WS_HR        object        
dtypes: datetime64[ns](1), fl

### 輸出光化變數與溫度 輸出

In [44]:
pic_cols =  [
    'SiteName', 
    'MonitorDate', 
    'AMB_TEMP', 
    'CH4', 
#     'CO', 
#     'CO2', 
    'NMHC', 
#     'NO',
#     'NO2', 
#     'NOx', 
#     'O3', 
#     'PH_RAIN', 
#     'PM10', 
#     'PM2.5', 
#     'RAINFALL', 
#     'RAIN_COND',
#     'RAIN_INT', 
#     'RH', 
#     'SO2', 
    'THC', 
#     'WD_HR', 
#     'WIND_DIREC', 
#     'WIND_SPEED',
#     'WS_HR'
]
name = f"光化變數與溫度_2018to2021_06"
formtoPkl(df_pivot.loc[:, pic_cols], name)

 ==輸出備份pkl== ::: 光化變數與溫度_2018to2021_06_20210709_18.29.03.pkl


PM2.5客製 3小時移動平均(X)、日平均(Y)

In [22]:
select_item = ["PM2.5"]
# 排序
df.sort_values(by=["SiteName", "DataCreationDate"], ascending=[1,1], inplace=True)
df.head()

,index,SiteName,County,AQI,Pollutant,Status,SO2,CO,O3,O3_8hr,...,WindDirec,DataCreationDate,Unit,CO_8hr,PM2.5_AVG,PM10_AVG,SO2_AVG,Longitude,Latitude,SiteId
3009307,26,三義,苗栗縣,19.0,None,良好,1.6,0.23,43.0,22.0,...,46.0,2016-11-25 13:00:00.000000,None,0.23,2.0,15,None,NaN,NaN,NaN
3009410,129,三義,苗栗縣,24.0,None,良好,1.5,0.26,57.0,28.0,...,35.0,2016-11-25 14:00:00.000000,None,0.23,2.0,15,None,NaN,NaN,NaN
3009551,270,三義,苗栗縣,30.0,None,良好,1.7,0.29,66.0,35.0,...,55.0,2016-11-25 15:00:00.000000,None,0.24,3.0,16,None,NaN,NaN,NaN
3009626,345,三義,苗栗縣,34.0,None,良好,2.0,0.31,46.0,40.0,...,32.0,2016-11-25 16:00:00.000000,None,0.25,3.0,17,None,NaN,NaN,NaN
3009772,491,三義,苗栗縣,36.0,None,良好,2.0,0.34,40.0,42.0,...,71.0,2016-11-25 17:00:00.000000,None,0.26,4.0,18,None,NaN,NaN,NaN


In [23]:
# 3小時移動平均(X)，可容忍3缺1筆
for col in select_item:
    col_name = col+"_MA3"
    df.loc[:, col_name] = df.sort_values(by=["SiteName", "DataCreationDate"], ascending=[1,1]).groupby(
                                'SiteName')[col].rolling(
                                window=3,min_periods=2).mean().reset_index().set_index("level_1")[col].apply(
                                lambda m: int(m) if pd.notna(m) else np.nan )

In [24]:
# 24小時移動平均(y)，可容忍24缺12筆
for col in select_item:
    col_name = col+"_MA24"
    
    df.loc[:, col_name] = df.sort_values(by=["SiteName", "DataCreationDate"], ascending=[1,1]).groupby(
                                'SiteName')[col].rolling(
                                window=24,min_periods=12).mean().reset_index().set_index("level_1")[col].apply(
                                lambda m: int(m) if pd.notna(m) else np.nan )

In [9]:
# 日平均值
df_PM25_DateAvg = df.loc[:, ["SiteName","date", "PM2.5"]].groupby(["SiteName","date"], as_index=False).mean()
df_PM25_DateAvg.rename(columns={"PM2.5":"PM2.5_DateAvg"}, inplace=True)
df_PM25_DateAvg.loc[:,"PM2.5_DateAvg"]=df_PM25_DateAvg.loc[:,"PM2.5_DateAvg"].apply(
                                    lambda m: int(m) if pd.notna(m) else np.nan )
df = df.merge(df_PM25_DateAvg, how="left", on=['SiteName', "date"])

In [26]:
# (option)

## 0~6點平均值
# mask_0to6 = df.loc[:,"DataCreationDate"].dt.hour <=6
# df_PM25_DateAvg_0to6 = df.loc[mask_0to6, ["SiteName","date", "PM2.5"]].groupby(["SiteName","date"], as_index=False).mean()
# df_PM25_DateAvg_0to6.rename(columns={"PM2.5":"PM2.5_DateAvg_0to6"}, inplace=True)
# df_PM25_DateAvg_0to6.loc[:,"PM2.5_DateAvg_0to6"]=df_PM25_DateAvg_0to6.loc[:,"PM2.5_DateAvg_0to6"].apply(
#                                     lambda m: int(m) if pd.notna(m) else np.nan )
# df = df.merge(df_PM25_DateAvg_0to6, how="left", on=['SiteName', "date"])

# # 0~11點平均值
# mask_0to11 = df.loc[:,"DataCreationDate"].dt.hour <=11
# df_PM25_DateAvg_0to11 = df.loc[mask_0to11, ["SiteName","date", "PM2.5"]].groupby(["SiteName","date"], as_index=False).mean()
# df_PM25_DateAvg_0to11.rename(columns={"PM2.5":"PM2.5_DateAvg_0to11"}, inplace=True)
# df_PM25_DateAvg_0to11.loc[:,"PM2.5_DateAvg_0to11"]=df_PM25_DateAvg_0to11.loc[:,"PM2.5_DateAvg_0to11"].apply(
#                                     lambda m: int(m) if pd.notna(m) else np.nan )
# df = df.merge(df_PM25_DateAvg_0to11, how="left", on=['SiteName', "date"])

# # 每日小時值最大值
# df_PM25_daliy_max = df.loc[:, ["SiteName","date", "PM2.5"]].groupby(["SiteName","date"], as_index=False).max()
# df_PM25_daliy_max.rename(columns={"PM2.5":"PM2.5_max"}, inplace=True)
# df_PM25_daliy_max.loc[:,"PM2.5_max"]=df_PM25_daliy_max.loc[:,"PM2.5_max"].apply(
#                                     lambda m: int(m) if pd.notna(m) else np.nan )
# df = df.merge(df_PM25_daliy_max, how="left", on=['SiteName', "date"])


# # 4+12每日最大值
# df_PM25_avg_max = df.loc[:, ["SiteName","date", "PM2.5_AVG"]].groupby(["SiteName","date"], as_index=False).max()
# df_PM25_avg_max.rename(columns={"PM2.5_AVG":"PM2.5_AVG_max"}, inplace=True)
# df_PM25_avg_max.loc[:,"PM2.5_AVG_max"]=df_PM25_avg_max.loc[:,"PM2.5_AVG_max"].apply(
#                                     lambda m: int(m) if pd.notna(m) else np.nan )
# df = df.merge(df_PM25_avg_max, how="left", on=['SiteName', "date"])

# # MA3每日最大值
# df_PM25_MA3_max = df.loc[:, ["SiteName","date", "PM2.5_MA3"]].groupby(["SiteName","date"], as_index=False).max()
# df_PM25_MA3_max.rename(columns={"PM2.5_MA3":"PM2.5_MA3_max"}, inplace=True)
# df_PM25_MA3_max.loc[:,"PM2.5_MA3_max"]=df_PM25_MA3_max.loc[:,"PM2.5_MA3_max"].apply(
#                                     lambda m: int(m) if pd.notna(m) else np.nan )
# df = df.merge(df_PM25_MA3_max, how="left", on=['SiteName', "date"])

# # MA24每日最大值
# df_PM25_MA24_max = df.loc[:, ["SiteName","date", "PM2.5_MA24"]].groupby(["SiteName","date"], as_index=False).max()
# df_PM25_MA24_max.rename(columns={"PM2.5_MA24":"PM2.5_MA24_max"}, inplace=True)
# df_PM25_MA24_max.loc[:,"PM2.5_MA24"]=df_PM25_MA24_max.loc[:,"PM2.5_MA24_max"].apply(
#                                     lambda m: int(m) if pd.notna(m) else np.nan )
# df = df.merge(df_PM25_MA24_max, how="left", on=['SiteName', "date"])

In [27]:
df.columns

Index(['index', 'SiteName', 'County', 'AQI', 'Pollutant', 'Status', 'SO2',
       'CO', 'O3', 'O3_8hr', 'PM10', 'PM2.5', 'NO2', 'NOx', 'NO', 'WindSpeed',
       'WindDirec', 'DataCreationDate', 'Unit', 'CO_8hr', 'PM2.5_AVG',
       'PM10_AVG', 'SO2_AVG', 'Longitude', 'Latitude', 'SiteId', 'PM2.5_MA3',
       'PM2.5_MA24', 'date', 'PM2.5_DateAvg'],
      dtype='object')

In [28]:
# df.drop(columns=["PM2.5_DateAvg_0to6_x","PM2.5_DateAvg_0to11_x"], inplace=True)
# df.rename(columns={"PM2.5_DateAvg_0to6_y":"PM2.5_DateAvg_0to6","PM2.5_DateAvg_0to11_y":"PM2.5_DateAvg_0to11"}, inplace=True)

In [30]:
# # 相關係數圖 1
# df_for_plot = df[["PM2.5","PM2.5_DateAvg_0to6","PM2.5_DateAvg_0to11", "PM2.5_MA3", "PM2.5_AVG","PM2.5_MA24", "PM2.5_DateAvg"]].copy(deep=True)
# df_for_plot.rename(columns={"PM2.5_AVG":"PM2.5_4+12"}, inplace=True)
# corr = df_for_plot.corr()
# corr.style.background_gradient(cmap='coolwarm')

In [31]:
# # 相關係數圖 2
# df_for_plot_2 = df[["PM2.5_max", "PM2.5_MA3_max", "PM2.5_AVG_max","PM2.5_MA24_max", "PM2.5_DateAvg"]].copy(deep=True)
# df_for_plot_2.rename(columns={"PM2.5_AVG_max":"PM2.5_4+12_max"}, inplace=True)
# corr = df_for_plot_2.corr()
# corr.style.background_gradient(cmap='coolwarm')

In [34]:
# df[["PM2.5", "PM2.5_AVG", "PM2.5_MA3", "PM2.5_MA24", "PM2.5_DateAvg"]].tail()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522511 entries, 0 to 3522510
Data columns (total 27 columns):
 #   Column            Dtype         
---  ------            -----         
 0   index             int64         
 1   SiteName          object        
 2   County            object        
 3   AQI               float64       
 4   Pollutant         object        
 5   Status            object        
 6   SO2               object        
 7   CO                float64       
 8   O3                float64       
 9   O3_8hr            float64       
 10  PM10              object        
 11  PM2.5             object        
 12  NO2               object        
 13  NOx               float64       
 14  NO                object        
 15  WindSpeed         object        
 16  WindDirec         object        
 17  DataCreationDate  datetime64[ns]
 18  Unit              object        
 19  CO_8hr            float64       
 20  PM2.5_AVG         object        
 21  PM10_AVG

In [36]:
print(df.date.max(), "\n", df.date.min())

2021-05-29 
 2016-11-25


In [10]:
df.head()

,index,SiteName,County,AQI,Pollutant,Status,SO2,CO,O3,O3_8hr,...,DataCreationDate,Unit,CO_8hr,PM2.5_AVG,PM10_AVG,SO2_AVG,Longitude,Latitude,SiteId,date
0,0,基隆,基隆市,38.0,None,良好,1.7,0.0,34.0,34.0,...,2021-01-01 04:00:00,None,0.2,12.0,26,2.0,121.760056,25.129167,1.0,2021-01-01
1,1,汐止,新北市,50.0,None,良好,0.5,0.0,23.0,25.0,...,2021-01-01 04:00:00,None,0.3,15.0,25,1.0,121.642300,25.067131,2.0,2021-01-01
2,2,萬里,新北市,56.0,懸浮微粒,普通,1.2,0.0,NaN,NaN,...,2021-01-01 04:00:00,None,0.2,16.0,62,3.0,121.689881,25.179667,3.0,2021-01-01
3,3,新店,新北市,42.0,None,良好,1.6,0.0,29.0,24.0,...,2021-01-01 04:00:00,None,0.3,13.0,23,2.0,121.537778,24.977222,4.0,2021-01-01
4,4,土城,新北市,46.0,None,良好,1.1,0.0,28.0,25.0,...,2021-01-01 04:00:00,None,0.3,14.0,24,2.0,121.451861,24.982528,5.0,2021-01-01


In [8]:
# 保存
formtoPkl(df, 'AQI_rowdata')

# 讀取
# raw_path = r'G:\我的雲端硬碟\python\output\0209\AQI_rowdata_20210209_14.02.56.pkl'
# df = read_data_dir_fPathExt(raw_path)

 ==輸出備份pkl== ::: AQI_rowdata_20210705_11.21.18.pkl


# 3.資料處理(AQI不用)

#### 照日期排序、刪除重複

In [9]:
date_col = "DataCreationDate"
# 'MonitorDate'
df.sort_values(by=['SiteName',date_col], inplace=True)
df.drop_duplicates(keep="first", inplace=True)
df.reset_index(drop=True, inplace=True)
# df.tail()

#### 篩選需要的欄位

In [12]:
df_pic = df.loc[:,['SiteName', 'MonitorDate', 'MonitorValue']].copy(deep=True)
df_pic.rename(columns={'MonitorValue':select_item}, inplace=True)
# df_pic.head()

#### (累積) 清理數值

In [105]:
bad_lst = ['x']
df_pic.loc[:,select_item] = df_pic.loc[:,select_item].apply(lambda v: np.nan if v in bad_lst else float(v) )

#### (個別處理) 製作平均值指標

1. O3_8hr = O3前8小時移動平均值 (不含當下小時值)
2. PM25_avg = 0.5 × 前 12 小時平均 + 0.5 × 前 4 小時平均 (不含當下小時值)

#### (遺漏值補植)新增欄位 針對小時值遺漏值進行線性差補(最多連續5個nan) ex: 1,nan,nan,7 >> 1,3,5,7

In [106]:
times = pd.date_range(start=df_pic.MonitorDate.min(), end=df_pic.MonitorDate.max(), freq='H')
df_times = pd.DataFrame(times, columns=['MonitorDate'])

if select_item=='O3':

    df_O3_8hr = pd.DataFrame()
    for s in df_pic['SiteName'].unique():
    # for s in ['三義']:
    #     填補缺漏的日期紀錄
        df_tmp = df_pic.loc[df_pic['SiteName'].isin([s]),['MonitorDate', 'O3']].copy(deep=True)
        df_tmp = df_times.merge(df_tmp, how='left', on='MonitorDate').set_index('MonitorDate')

    # #     非數值轉換為np.nan
        df_tmp.loc[:,'O3'] = df_tmp.loc[:,'O3'].apply(lambda v: v if isinstance(v, (int,float)) else np.nan)
        
#         (遺漏值補植)針對小時值遺漏值進行線性差補 ex: 1,nan,nan,7 >> 1,3,5,7
        df_tmp['O3_interp']  = df_tmp['O3'].interpolate(limit=5)
        
    # #     取前8hr平均，不包含自己(shift 1 格)
        df_tmp = df_tmp.shift(1).rolling(8).mean() 
        df_tmp['SiteName'] = s
        df_tmp.reset_index(inplace=True)
        df_O3_8hr = pd.concat([df_O3_8hr,df_tmp])

    df_O3_8hr.rename(columns={'O3':'O3_8hr', 'O3_interp':'O3_interp_8hr'}, inplace=True)
    
    
if select_item=='PM2.5':
 
    df_PM25_avg = pd.DataFrame()
    for s in df_pic['SiteName'].unique():
    # for s in ['三義']:
    #     填補缺漏的日期紀錄
        df_tmp = df_pic.loc[df_pic['SiteName'].isin([s]),['MonitorDate', 'PM2.5']].copy(deep=True)
        df_tmp = df_times.merge(df_tmp, how='left', on='MonitorDate').set_index('MonitorDate')

    # #     非數值轉換為np.nan
        df_tmp.loc[:,'PM2.5'] = df_tmp.loc[:,'PM2.5'].apply(lambda v: v if isinstance(v, (int,float)) else np.nan)
        
#         (遺漏值補植)針對小時值遺漏值進行線性差補 ex: 1,nan,nan,7 >> 1,3,5,7
        df_tmp['PM2.5_interp']  = df_tmp['PM2.5'].interpolate(limit=5)
    
#     取前12hr平均、前4hr平均，各佔50%，不包含自己(shift 1 格)
        df_tmp_12hr = df_tmp.shift(1).rolling(12).mean()
        df_tmp_4hr = df_tmp.shift(1).rolling(4).mean()
        df_tmp_all = (df_tmp_12hr*0.5) +  (df_tmp_4hr*0.5)
        df_tmp_all['SiteName'] = s
        df_tmp_all.reset_index(inplace=True)
        df_PM25_avg = pd.concat([df_PM25_avg, df_tmp_all])

    df_PM25_avg.rename(columns={'PM2.5':'PM2.5_avg', 'PM2.5_interp':'PM2.5_interp_avg'}, inplace=True)

In [107]:
# (遺漏值補植) 補回 小時值本身的插植結果 (最多連續5個nan)
if select_item=='O3':
    df_pic['O3_interp']  = df_pic['O3'].interpolate(limit=5)
if select_item=='PM2.5':
    df_pic['PM2.5_interp']  = df_pic['PM2.5'].interpolate()

In [108]:
key_cols = ['SiteName', 'MonitorDate']
#統一時間長度
if select_item=='O3':
    df_pic = df_O3_8hr[key_cols].merge(df_pic, how='left',on=key_cols)
    df_result = df_pic.merge(df_O3_8hr, how='left', on=key_cols)
    
if select_item=='PM2.5':
    df_pic = df_PM25_avg[key_cols].merge(df_pic, how='left',on=key_cols)
    df_result = df_pic.merge(df_PM25_avg, how='left', on=key_cols)

In [109]:
df_result.head()

,SiteName,MonitorDate,PM2.5,PM2.5_interp,PM2.5_avg,PM2.5_interp_avg
0,三義,2009-12-31 00:00:00,42.0,42.0,NaN,NaN
1,三義,2009-12-31 01:00:00,46.0,46.0,NaN,NaN
2,三義,2009-12-31 02:00:00,51.0,51.0,NaN,NaN
3,三義,2009-12-31 03:00:00,50.0,50.0,NaN,NaN
4,三義,2009-12-31 04:00:00,52.0,52.0,NaN,NaN


In [112]:
df_result['SiteName'].nunique()

77

#### 存檔

In [75]:
# mask = (df_result['SiteName'].isin(['三義'])) & (df_result['MonitorDate'].dt.date.isin([date(2020, 1, 15)]))
# df_result[mask].sort_values(by=['MonitorDate'])

In [110]:
if select_item=='O3':
    name = f"O3小時值_{df_result['MonitorDate'].dt.date.min()}to{df_result['MonitorDate'].dt.date.max()}"
    formtoCsv(df_result, name)
    
if select_item=='PM2.5':
    name = f"PM2.5小時值_{df_result['MonitorDate'].dt.date.min()}to{df_result['MonitorDate'].dt.date.max()}"
    formtoCsv(df_result, name)

 ==輸出備份csv==::: PM2.5小時值_2009-12-31to2021-01-31_20210204_16.36.14.csv
